# 01. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Import Data

In [2]:
path = r'C:\Users\esteb\OneDrive\Desktop\2025_Instacart Basket Analysis'

In [3]:
df_new_cus = pd.read_csv(os.path.join(path, '02_Data', 'Original Data', 'customers.csv'))

In [4]:
df_agg = pd.read_pickle(os.path.join(path, '02_Data', 'Prepared Data', 'aggregated_ords_prods_merge.pkl'))

# 03. Wrangle the Data

In [5]:
# Fundamental data quality and consistency check
df_new_cus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [6]:
# Fundamental data quality and consistency check
df_new_cus.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
# Drop unecessary column
df_new_cus = df_new_cus.drop('date_joined', axis = 1)

In [8]:
# Check dataframe for dropped columns
df_new_cus.head()

,user_id,First Name,Surnam,Gender,STATE,Age,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1,married,40374


In [9]:
# Renaming Columns
df_cus = df_new_cus.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age'})

In [10]:
# Check Renamed Columns
df_cus.head()

,user_id,first_name,last_name,gender,state,age,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1,married,40374


In [11]:
# Count the Missing values per column
df_cus.isnull().sum()

user_id             0
first_name      11259
last_name           0
gender              0
state               0
age                 0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [12]:
# Handle missing values by creating a full_name column. 
df_cus['full_name'] = df_cus['first_name'].fillna('') + ' ' + df_cus['last_name']

In [13]:
# If Nan first name in full_name column then use last name
df_cus['full_name'] = df_cus['full_name'].str.strip()

In [14]:
#Drop first_name and last_name columns
df_cus.drop(['first_name', 'last_name'], axis=1, inplace=True)

In [16]:
# View Data Types
df_cus.dtypes

user_id          int64
gender          object
state           object
age              int64
n_dependants     int64
fam_status      object
income           int64
full_name       object
dtype: object

In [15]:
# Check for duplicates
df_cus.duplicated().sum()

0

In [17]:
# Check for mixed Data Types
df_cus.applymap(type).nunique()

C:\Users\esteb\AppData\Local\Temp\ipykernel_36528\2476940742.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cus.applymap(type).nunique()


user_id         1
gender          1
state           1
age             1
n_dependants    1
fam_status      1
income          1
full_name       1
dtype: int64

In [18]:
# Check Wrangled Work
df_cus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   gender        206209 non-null  object
 2   state         206209 non-null  object
 3   age           206209 non-null  int64 
 4   n_dependants  206209 non-null  int64 
 5   fam_status    206209 non-null  object
 6   income        206209 non-null  int64 
 7   full_name     206209 non-null  object
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


# 04. Combine Datasets

In [19]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  daily_frequency         object 
 15  new_daily_frequency     object 
 16  busiest_period_of_day   object 
 17  max_order               int64  
 18  loyalty_flag            object 
 19  average_spend_item      float64
 20  spnd_itm_price_flag     object 
 21  total_order_spend       float

The common column is the user_id

In [20]:
# Use merge function to combine datasets
df = df_agg.merge(df_cus, on = ['user_id'], how = 'left')

Adds all customer data to df_agg without loosing any information from df_agg. The left join allows all rows from df_agg to be match with customer info from df_cus where possible.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 32 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  daily_frequency         object 
 15  new_daily_frequency     object 
 16  busiest_period_of_day   object 
 17  max_order               int64  
 18  loyalty_flag            object 
 19  average_spend_item      float64
 20  spnd_itm_price_flag     object 
 21  total_order_spend       float

Merge seems to be succesful.

In [22]:
# Count the Missing values per column
df.isnull().sum()

order_id                        0
user_id                         0
order_number                    0
orders_day_of_week              0
order_hour_of_day               0
days_since_prior_order    2076096
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
prices                          0
price_range_loc                 0
daily_frequency                 0
new_daily_frequency             0
busiest_period_of_day           0
max_order                       0
loyalty_flag                    0
average_spend_item              0
spnd_itm_price_flag             0
total_order_spend               0
order_spend_flag                0
customer_freq                   5
customer_freq_flag              5
gender                          0
state                           0
age                             0
n_dependants                    0
fam_status    

In [23]:
# Check for missing values in customer_freq and customer_freq_flag
df[df[['customer_freq', 'customer_freq_flag']].isnull().any(axis=1)]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_spend_flag,customer_freq,customer_freq_flag,gender,state,age,n_dependants,fam_status,income,full_name
25156870,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156871,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156872,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156873,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156874,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava


In [24]:
# Show user to deep dive into frequency
df[df['user_id'] == 159838]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_spend_flag,customer_freq,customer_freq_flag,gender,state,age,n_dependants,fam_status,income,full_name
25156870,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156871,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156872,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156873,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava
25156874,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Lower Spender,NaN,NaN,Male,Arkansas,63,3,married,140994,Jason Nava


Seems like this user only used instacart 1 time and should have a frequency of 0.0 and a frequency flag of Non-Frequent Customer

In [25]:
# Change all the values in the customer_freq column of user_id 159838
df.loc[df['user_id'] == 159838, 'customer_freq'] = 0.0

In [26]:
# Change all the values in the customer_freq_flag column of user_id 159838
df.loc[df['user_id'] == 159838, 'customer_freq_flag'] = 'Non-Frequent Customer'

In [27]:
# Check for mixed Data Types
df.apply(lambda col: col.map(type).nunique())

order_id                  1
user_id                   1
order_number              1
orders_day_of_week        1
order_hour_of_day         1
days_since_prior_order    1
product_id                1
add_to_cart_order         1
reordered                 1
product_name              1
aisle_id                  1
department_id             1
prices                    1
price_range_loc           1
daily_frequency           1
new_daily_frequency       1
busiest_period_of_day     1
max_order                 1
loyalty_flag              1
average_spend_item        1
spnd_itm_price_flag       1
total_order_spend         1
order_spend_flag          1
customer_freq             1
customer_freq_flag        1
gender                    1
state                     1
age                       1
n_dependants              1
fam_status                1
income                    1
full_name                 1
dtype: int64

In [29]:
# Count the Missing values per column again
df.isnull().sum()

order_id                        0
user_id                         0
order_number                    0
orders_day_of_week              0
order_hour_of_day               0
days_since_prior_order    2076096
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
prices                          0
price_range_loc                 0
daily_frequency                 0
new_daily_frequency             0
busiest_period_of_day           0
max_order                       0
loyalty_flag                    0
average_spend_item              0
spnd_itm_price_flag             0
total_order_spend               0
order_spend_flag                0
customer_freq                   0
customer_freq_flag              0
gender                          0
state                           0
age                             0
n_dependants                    0
fam_status    

# Export Data

In [31]:
df.to_pickle(os.path.join(path, '02_Data', 'Prepared Data', 'instacart_dataset.pkl'))